In [2]:

############################################## 00. 필요한 파이썬 라이브러리 불러오기 #####################################################
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import folium
import webbrowser

import sys
import plotly.express as px
from numpy import random
import missingno

from sklearn.preprocessing import StandardScaler, normalize
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn import tree 
from sklearn.tree import DecisionTreeClassifier

import spark
import pydot 
from IPython.display import Image
import pyarrow as pa

import json
import urllib
import math
from tqdm import tqdm
import warnings

################### 설정 ########################
pd.options.display.float_format = '{:.10f}'.format

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [3]:
# 01~06월 데이터 

bike_df = pd.DataFrame()

month_list = [str(i).zfill(2) for i in range(1,7)]
for i in month_list:
    data = pd.read_csv("공공자전거 대여이력 정보_21"+ i +".csv",encoding='cp949',low_memory=False)
    bike_df = pd.concat((bike_df,data),sort=False)
    print(data.head())
    


       자전거번호                 대여일시  대여 대여소번호 대여 대여소명  대여거치대  \
0  SPB-53145  2021-01-02 20:50:36         3    중랑센터      0   
1  SPB-53074  2021-01-04 16:02:12         3    중랑센터      0   
2  SPB-50952  2021-01-13 21:02:05         3    중랑센터      0   
3  SPB-52380  2021-01-14 21:02:55         3    중랑센터      0   
4  SPB-52380  2021-01-18 18:02:08         3    중랑센터      0   

                  반납일시  반납대여소번호           반납대여소명  반납거치대  이용시간            이용거리  
0  2021-01-02 21:15:41      668     서울축산농협(장안지점)      0    25    0.0000000000  
1  2021-01-04 16:17:06      668     서울축산농협(장안지점)      0    14    0.0000000000  
2  2021-01-13 21:14:04      668     서울축산농협(장안지점)      0    11 1853.9900000000  
3  2021-01-14 21:13:41      668     서울축산농협(장안지점)      0    10    0.0000000000  
4  2021-01-18 18:15:22      540  군자역 7번출구 베스트샵 앞      0    13 2291.0500000000  
       자전거번호                 대여일시  대여 대여소번호           대여 대여소명 대여거치대  \
0  SPB-51323  2021-02-01 00:00:38      1514  강북구청 사거리 버스정류소 앞     0   
1  SP

In [4]:
bike_df.isnull().sum()

자전거번호         0
대여일시          0
대여 대여소번호      0
대여 대여소명       0
대여거치대         0
반납일시          0
반납대여소번호       0
반납대여소명        0
반납거치대         0
이용시간          0
이용거리        269
dtype: int64

- 이용거리만 결측치가 있음.

In [5]:
#이용거리에 결측값 있는 데이터 269건 출력

bike_df[bike_df["이용거리"].isnull()]


,자전거번호,대여일시,대여 대여소번호,대여 대여소명,대여거치대,반납일시,반납대여소번호,반납대여소명,반납거치대,이용시간,이용거리
32787,SPB-39268,2021-02-01 19:12:46,567,성수역 2번출구 앞,0,2021-02-01 19:45:48,00544,"광남?峠閨?,0""",33,6717.9100000000,NaN
152340,SPB-30921,2021-02-05 19:38:47,2729,넥센 유니버시티 앞,0,2021-02-05 19:58:40,01175,"대한항공 ?管째낱鈒씽?,0""",19,3507.1400000000,NaN
164858,SPB-37148,2021-02-06 09:13:04,2036,"정금마을 ?뗌뻘治봐ㅇ昰?,0""",2021-02-06 09:29:07,02221,하나은행 방배동지점 앞,0,16,1857.9200000000,NaN
235075,SPB-51619,2021-02-07 21:54:29,1911,구로디지털단지역 앞,0,2021-02-07 21:59:17,01856,"모?括피閨?,0""",4,984.8900000000,NaN
242801,SPB-38736,2021-02-08 08:54:32,164,"북가좌1동 주민센??,0""",2021-02-08 08:57:41,03123,DMC파크뷰자이1단지 110동 앞,0,3,570.5900000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...
3127782,SPB-48293,2021-06-28 09:29:38,1020,강동경찰서,0,2021-06-28 09:37:55,02622,"올림픽공원역 3번출??,0""",8,1660.0000000000,NaN
3158679,SPB-38582,2021-06-28 17:08:17,1462,동부시장 북문 앞,0,2021-06-28 17:21:54,00670,"삼육서울병원 버스정?昰?,0""",13,1471.8400000000,NaN
3334062,SPB-52358,2021-06-30 07:02:14,1438,홈플러스 신내점 앞,0,2021-06-30 07:10:26,01436,"영풍?떫藥뭔?,0""",8,0.0000000000,NaN
3369842,SPB-39990,2021-06-30 14:11:21,4152,"상도효성해?돼逑첨뮌決?,0""",2021-06-30 14:22:44,04175,스페이스 살림 앞,0,11,2209.2000000000,NaN


In [ ]:
# 01~06월 데이터 
def wrongdata():
    month_list = [str(i).zfill(2) for i in range(1,7)]
    for i in month_list:
        df = pd.read_csv("공공자전거 대여이력 정보_21"+ i +".csv",encoding='cp949',low_memory=False)
        print(df["대여거치대"].unique())

wrongdata()

[ 0  7 99  1  4  3 19  2 20 16 11 12 10  9  5  8  6 15 23 17 13 14 18 40
 38 33 22 24 39 26 21 27 25 32 29 30 34 31 28 36 37 35]
['0' '8' '99' '1' '7' '11' '6' '2' '10' '5' '17' '3' '4' '15' '16' '12'
 '9' '18' '19' '14' '13' '21' '28' '20' '26' '30' '25' '22' '23' '24' '39'
 '36' '37' '34' '32' '29' '27' '2021-02-06 09:29:07' '31' '33'
 '2021-02-08 08:57:41' '2021-02-09 22:11:59' '35' '2021-02-18 13:04:47'
 '40' '2021-02-22 15:03:56' '38' '2021-02-24 08:11:15'
 '2021-02-26 12:32:20' '2021-02-26 13:47:05' '2021-02-27 21:07:10']
['0' '4' '99' '9' '3' '10' '17' '5' '2' '8' '7' '1' '13' '12' '11' '14'
 '6' '18' '20' '16' '15' '19' '2021-03-02 14:20:36' '29' '23'
 '2021-03-02 20:39:35' '2021-03-02 22:30:53' '30' '28'
 '2021-03-03 18:08:05' '25' '39' '2021-03-03 19:10:48' '33' '21'
 '2021-03-04 06:37:43' '26' '2021-03-04 12:53:15' '27'
 '2021-03-05 15:47:47' '2021-03-06 00:22:21' '24' '34'
 '2021-03-06 14:09:35' '22' '2021-03-08 14:32:51' '37' '40'
 '2021-03-08 18:17:54' '2021-03-09 07:23:1

- 요약: 21년 2월 데이터부터, 밀려 있는 데이터가 하나씩 존재함.

>> 결론 : 이용거리가 NaN인 데이터 삭제

In [9]:
new_bike_df = bike_df[~bike_df["이용거리"].isnull()]

In [11]:
new_bike_df

,자전거번호,대여일시,대여 대여소번호,대여 대여소명,대여거치대,반납일시,반납대여소번호,반납대여소명,반납거치대,이용시간,이용거리
0,SPB-53145,2021-01-02 20:50:36,3,중랑센터,0,2021-01-02 21:15:41,668,서울축산농협(장안지점),0,25.0000000000,0.0000000000
1,SPB-53074,2021-01-04 16:02:12,3,중랑센터,0,2021-01-04 16:17:06,668,서울축산농협(장안지점),0,14.0000000000,0.0000000000
2,SPB-50952,2021-01-13 21:02:05,3,중랑센터,0,2021-01-13 21:14:04,668,서울축산농협(장안지점),0,11.0000000000,1853.9900000000
3,SPB-52380,2021-01-14 21:02:55,3,중랑센터,0,2021-01-14 21:13:41,668,서울축산농협(장안지점),0,10.0000000000,0.0000000000
4,SPB-52380,2021-01-18 18:02:08,3,중랑센터,0,2021-01-18 18:15:22,540,군자역 7번출구 베스트샵 앞,0,13.0000000000,2291.0500000000
...,...,...,...,...,...,...,...,...,...,...,...
3445939,SPB-36886,2021-06-30 22:44:14,152,마포구민체육센터 앞,0,2021-07-01 04:48:33,00126,서강대 후문 옆,0,364.0000000000,10153.3600000000
3445940,SPB-30409,2021-06-30 22:45:19,152,마포구민체육센터 앞,0,2021-07-01 04:52:25,00437,대흥역 1번출구,0,367.0000000000,10100.7100000000
3445941,SPB-46230,2021-06-30 22:43:43,152,마포구민체육센터 앞,0,2021-07-01 04:52:27,00437,대흥역 1번출구,0,368.0000000000,430.0000000000
3445942,SPB-41691,2021-06-30 22:11:14,2220,반포본동 주민센터 앞,0,2021-07-01 04:55:29,02526,반포경남쇼핑 앞,0,404.0000000000,0.0000000000


: 